In [ ]:
import json

In [1]:
import json

# Read the jsonl file and convert it to a JSON list
def jsonl_to_json_list(jsonl_file_path):
    json_list = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as file:
        for line in file:
            json_obj = json.loads(line.strip())  # Parse each line as JSON
            json_list.append(json_obj)
    
    return json_list

# Save the JSON list to a file
def save_as_json(json_list, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as outfile:
        json.dump(json_list, outfile, indent=4)

# Example usage:
json_list = jsonl_to_json_list('/data/jiwoong/workspace/MDAgents/data/medqa.jsonl')
save_as_json(json_list, '/data/jiwoong/workspace/MDAgents/data/medqa.json')
